##**ASSIGNMENT-2** *LINEAR REGRESSION* on netflix stock price dataset from kaggle.
###DIVYA S GUPTA RA1911003010454 CSE-H1

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Build a linear regression  based on the target variable.The assignment should include k-fold model validation, the suitable metric for evaluation and comparison,etc.**
####Here the target variable in 'Open'

**Importing dataset from drive.** 

In [2]:
# required library for visual and stats 
import numpy as np
import pandas as pd #dataset
import seaborn as sns #for graphic effects
import matplotlib.pyplot as plt #to plot
import scipy.stats as stats #statistics

In [3]:
ds=pd.read_csv(r'/content/gdrive/MyDrive/ml datasets/Netflix.csv') #imported
print(ds)

            Date        Open        High  ...       Close   Adj Close     Volume
0     2002-05-23    1.156429    1.242857  ...    1.196429    1.196429  104790000
1     2002-05-24    1.214286    1.225000  ...    1.210000    1.210000   11104800
2     2002-05-28    1.213571    1.232143  ...    1.157143    1.157143    6609400
3     2002-05-29    1.164286    1.164286  ...    1.103571    1.103571    6757800
4     2002-05-30    1.107857    1.107857  ...    1.071429    1.071429   10154200
...          ...         ...         ...  ...         ...         ...        ...
4576  2020-07-28  496.019989  497.790009  ...  488.510010  488.510010    5986700
4577  2020-07-29  492.250000  494.920013  ...  484.480011  484.480011    6611800
4578  2020-07-30  480.709991  488.350006  ...  485.799988  485.799988    6608600
4579  2020-07-31  488.290009  494.799988  ...  488.880005  488.880005    5924300
4580  2020-08-03  490.859985  503.089996  ...  498.619995  498.619995    5873100

[4581 rows x 7 columns]


In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4581 entries, 0 to 4580
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       4581 non-null   object 
 1   Open       4581 non-null   float64
 2   High       4581 non-null   float64
 3   Low        4581 non-null   float64
 4   Close      4581 non-null   float64
 5   Adj Close  4581 non-null   float64
 6   Volume     4581 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 250.6+ KB


In [5]:
ds["Date"]=pd.to_datetime(ds["Date"]) #date and time type
rd = 2 # round of to this many digit
ds["Open"]=ds["Open"].apply(lambda x: round(x,rd))
ds["High"]=ds["High"].apply(lambda x: round(x,rd))
ds["Low"]=ds["Low"].apply(lambda x: round(x,rd))
ds["Close"]=ds["Close"].apply(lambda x: round(x,rd))
ds["Adj Close"]=ds["Adj Close"].apply(lambda x: round(x,rd))
ds["Volume"]=ds["Volume"] / 1000
ds.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
dtype: object

data cleaning 

In [6]:
ds.isnull().sum() #to check if any null value is there 

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
ds[ds.duplicated(keep=False)] # to check for duplicate value

,Date,Open,High,Low,Close,Adj Close,Volume


data engineering

In [8]:
ds['Turnover']=((ds['High']+ds['Low'])/2)/((ds['Open']+ds['Close'])/2)*ds['Volume'] #new varaiable/feature 
print(ds)

           Date    Open    High  ...  Adj Close    Volume       Turnover
0    2002-05-23    1.16    1.24  ...       1.20  104790.0  106122.076271
1    2002-05-24    1.21    1.23  ...       1.21   11104.8   11150.687603
2    2002-05-28    1.21    1.23  ...       1.16    6609.4    6665.175527
3    2002-05-29    1.16    1.16  ...       1.10    6757.8    6727.898230
4    2002-05-30    1.11    1.11  ...       1.07   10154.2   10154.200000
...         ...     ...     ...  ...        ...       ...            ...
4576 2020-07-28  496.02  497.79  ...     488.51    5986.7    5992.902385
4577 2020-07-29  492.25  494.92  ...     484.48    6611.8    6627.504827
4578 2020-07-30  480.71  488.35  ...     485.80    6608.6    6604.155566
4579 2020-07-31  488.29  494.80  ...     488.88    5924.3    5937.213576
4580 2020-08-03  490.86  503.09  ...     498.62    5873.1    5895.892481

[4581 rows x 8 columns]


data selection

In [9]:
ds.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Turnover'],
      dtype='object')

In [10]:
#Let's select our features
x = ds.loc[:,'High':'Turnover'] 
# or X=ds[['High', 'Low', 'Close', 'Adj Close', 'Volume','Turnover']]
y = ds.loc[:,'Open']
# or Y=ds[['Open']]

In [11]:
x.head()

,High,Low,Close,Adj Close,Volume,Turnover
0,1.24,1.15,1.20,1.20,104790.0,106122.076271
1,1.23,1.20,1.21,1.21,11104.8,11150.687603
2,1.23,1.16,1.16,1.16,6609.4,6665.175527
3,1.16,1.09,1.10,1.10,6757.8,6727.898230
4,1.11,1.07,1.07,1.07,10154.2,10154.200000


In [12]:
y.head()

0    1.16
1    1.21
2    1.21
3    1.16
4    1.11
Name: Open, dtype: float64

split data in training and testing

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 0)

In [14]:
x_train

,High,Low,Close,Adj Close,Volume,Turnover
1643,3.26,3.11,3.15,3.15,7054.6,7110.411709
4168,281.77,266.48,274.88,274.88,11456.7,11577.276837
2106,22.68,21.65,22.34,22.34,47406.8,47546.231765
1191,3.24,3.20,3.21,3.21,8286.6,8312.414953
399,3.73,3.61,3.71,3.71,18925.2,18925.200000
...,...,...,...,...,...,...
1033,3.89,3.80,3.88,3.88,5491.5,5498.650391
3264,83.81,82.99,83.38,83.38,11907.0,11879.224834
1653,4.12,3.93,4.07,4.07,4821.6,4851.735000
2607,8.01,7.74,7.94,7.94,21350.7,21122.708857


In [15]:
y_train

1643      3.17
4168    267.66
2106     21.86
1191      3.21
399       3.63
         ...  
1033      3.80
3264     83.81
1653      3.93
2607      7.98
2732     26.27
Name: Open, Length: 3206, dtype: float64

linear regression modeling

In [16]:
from sklearn.linear_model import LinearRegression
lrm = LinearRegression()

In [17]:
lrm.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [18]:
lrm.score(x_test,y_test)

0.9998581140100695

In [19]:
pred=lrm.predict(x_test)
pred

array([16.77815656,  3.77394932, 94.64764214, ...,  0.71231229,
        3.52970988,  0.92005962])

In [20]:
accuracy = lrm.score(x_test,y_test)
print("accuracy = ", accuracy * 100, "%")

accuracy =  99.98581140100696 %


**Checking it on random rows**

In [48]:
import random
dscheck=ds.loc[[random.randint(0,2000)]]
dscheck['Open'] #actual value or observed value

550    2.87
Name: Open, dtype: float64

In [49]:
datac=np.asarray(dscheck.drop(['Date','Open'],1))
pp=lrm.predict(datac) #predicted value
pp

array([2.83607785])

k fold validation

In [21]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [22]:
kfold_cv = KFold(n_splits=10, random_state=45, shuffle=True)

In [23]:
model = LinearRegression()

In [24]:
kfold_scores = cross_val_score(model ,x_train, y_train, cv = kfold_cv)
kfold_scores

array([0.9997976 , 0.99988894, 0.99974054, 0.99986993, 0.99987937,
       0.99984055, 0.99988202, 0.99988116, 0.99977194, 0.99987736])

In [25]:
cross_validation_accuracy = np.mean(kfold_scores)
cross_validation_accuracy

0.9998429413916057

Mean Absolute Error (MAE)

In [26]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,pred)

0.5484166327562995

Mean Squared Error (MSE)

In [27]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,pred)

1.9370951422043736

Root Mean Squared Error (RMS)

In [28]:
np.sqrt(mean_squared_error(y_test,pred))

1.3917956538962082